In [1]:
import geopandas as gpd
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import pandas as pd
from shapely.ops import unary_union

In [3]:

# Load Shapefiles
shapefile1 = gpd.read_file("C:/Users/laura/Desktop/Housing/SocialHousing/data/raw/lor_planungsraeume_2021.shp")
shapefile2 = gpd.read_file("C:/Users/laura/Desktop/Housing/SocialHousing/data/raw/plz.shp")

# Set the geometry column name explicitly
shapefile1 = shapefile1.set_geometry('geometry')
shapefile2 = shapefile2.set_geometry('geometry')

shapefile2 = shapefile2[['plz', 'geometry' ]]

shapefile1['g1'] = shapefile1.geometry
shapefile2['g2'] = shapefile2.geometry
# Plotting the shapefiles 
# shapefile1.plot() - > Plots the second element 
# shapefile1.loc[1, 'geometry'] - > PLots the first element


In [4]:
inter = shapefile1.sjoin(shapefile2)

In [5]:
inter.head()

,PLR_ID,PLR_NAME,BEZ,STAND,GROESSE_M2,geometry,g1,index_right,plz,g2
0,11501341,Karlshorst Süd,11,01.01.2021,2294021.723,"POLYGON ((400826.929 5814871.126, 400825.707 5...","POLYGON ((400826.929 5814871.126, 400825.707 5...",101,12459,"POLYGON ((397915.738 5815858.337, 397915.742 5..."
8,11501238,Rummelsburg,11,01.01.2021,2401515.387,"POLYGON ((396519.569 5817953.766, 396549.601 5...","POLYGON ((396519.569 5817953.766, 396549.601 5...",101,12459,"POLYGON ((397915.738 5815858.337, 397915.742 5..."
40,09501939,Bahnhofstraße,09,01.01.2021,1142557.213,"POLYGON ((402902.008 5811425.146, 402902.231 5...","POLYGON ((402902.008 5811425.146, 402902.231 5...",101,12459,"POLYGON ((397915.738 5815858.337, 397915.742 5..."
72,09200818,Spindlersfeld,09,01.01.2021,571610.581,"POLYGON ((402026.490 5812420.943, 402027.950 5...","POLYGON ((402026.490 5812420.943, 402027.950 5...",101,12459,"POLYGON ((397915.738 5815858.337, 397915.742 5..."
131,11501340,Karlshorst Nord,11,01.01.2021,3297882.930,"POLYGON ((401689.815 5814706.189, 401617.432 5...","POLYGON ((401689.815 5814706.189, 401617.432 5...",101,12459,"POLYGON ((397915.738 5815858.337, 397915.742 5..."


In [6]:
def f(frame):
    """A function to calculate overlap percentage"""
    interpct = 100*frame.g1.intersection(frame.g2).area/frame.g2.area
    return interpct

inter['pct'] = inter.apply(f, axis=1)

def h(frame):
    """A function to calculate overlap percentage"""
    interpct = 100*frame.g1.intersection(frame.g2).area/frame.g1.area
    return interpct
inter['pct_plz'] = inter.apply(h, axis=1)

In [7]:
def g(frame): 
    area1 = frame.g1.intersection(frame.g2).area
    return area1 

def s(frame): 
    area2 = frame.g2.area
    return area2
    
inter ['area_intersect'] = inter.apply(g, axis=1)
inter ['area_PLRID'] = inter.apply(s, axis=1)

In [8]:
inter.head()

,PLR_ID,PLR_NAME,BEZ,STAND,GROESSE_M2,geometry,g1,index_right,plz,g2,pct,pct_plz,area_intersect,area_PLRID
0,11501341,Karlshorst Süd,11,01.01.2021,2294021.723,"POLYGON ((400826.929 5814871.126, 400825.707 5...","POLYGON ((400826.929 5814871.126, 400825.707 5...",101,12459,"POLYGON ((397915.738 5815858.337, 397915.742 5...",0.000109,0.000389,8.912544,8.198570e+06
8,11501238,Rummelsburg,11,01.01.2021,2401515.387,"POLYGON ((396519.569 5817953.766, 396549.601 5...","POLYGON ((396519.569 5817953.766, 396549.601 5...",101,12459,"POLYGON ((397915.738 5815858.337, 397915.742 5...",0.000035,0.000119,2.865780,8.198570e+06
40,09501939,Bahnhofstraße,09,01.01.2021,1142557.213,"POLYGON ((402902.008 5811425.146, 402902.231 5...","POLYGON ((402902.008 5811425.146, 402902.231 5...",101,12459,"POLYGON ((397915.738 5815858.337, 397915.742 5...",0.665961,4.781405,54599.263964,8.198570e+06
72,09200818,Spindlersfeld,09,01.01.2021,571610.581,"POLYGON ((402026.490 5812420.943, 402027.950 5...","POLYGON ((402026.490 5812420.943, 402027.950 5...",101,12459,"POLYGON ((397915.738 5815858.337, 397915.742 5...",0.000415,0.005956,34.027833,8.198570e+06
131,11501340,Karlshorst Nord,11,01.01.2021,3297882.930,"POLYGON ((401689.815 5814706.189, 401617.432 5...","POLYGON ((401689.815 5814706.189, 401617.432 5...",101,12459,"POLYGON ((397915.738 5815858.337, 397915.742 5...",0.469156,1.166977,38464.049287,8.198570e+06


In [11]:
# This is code to keep only those areas for which more then 50% was mached 
#inter_100 = inter.loc[inter['pct'] >= 50,:]
#len(inter_100)
#inter_100.plot()
#shapefile1_sub = shapefile1.loc[shapefile1['plz'] == '10785',:]
#shapefile2_sub = shapefile2.loc[shapefile2['PLR_ID'] == '01100102',:]

In [9]:
#final_areas = inter.groupby('PLR_ID', group_keys=False).apply(lambda x: x.loc[x.pct.idxmax()])
#len(final_areas)

2334

In [10]:
#final_areas.head()

,PLR_ID,PLR_NAME,BEZ,STAND,GROESSE_M2,geometry,g1,index_right,plz,g2,pct,pct_plz,area_intersect,area_PLRID,0
531,01100101,Stülerstraße,01,01.01.2021,366928.828,"POLYGON ((387952.642 5818274.558, 387924.234 5...","POLYGON ((387952.642 5818274.558, 387924.234 5...",45.0,10785,"POLYGON ((389813.704 5819026.681, 389810.807 5...",1.276860,10.626963,3.897417e+04,3.052346e+06,NaN
531,01100101,Stülerstraße,01,01.01.2021,366928.828,"POLYGON ((387952.642 5818274.558, 387924.234 5...","POLYGON ((387952.642 5818274.558, 387924.234 5...",46.0,10787,"POLYGON ((387973.509 5819508.755, 387994.142 5...",22.785279,89.373037,3.277738e+05,1.438533e+06,NaN
228,01100102,Großer Tiergarten,01,01.01.2021,3943375.966,"POLYGON ((389869.327 5820074.331, 389869.304 5...","POLYGON ((389869.327 5820074.331, 389869.304 5...",45.0,10785,"POLYGON ((389813.704 5819026.681, 389810.807 5...",61.175769,47.376183,1.867296e+06,3.052346e+06,NaN
228,01100102,Großer Tiergarten,01,01.01.2021,3943375.966,"POLYGON ((389869.327 5820074.331, 389869.304 5...","POLYGON ((389869.327 5820074.331, 389869.304 5...",46.0,10787,"POLYGON ((387973.509 5819508.755, 387994.142 5...",56.398230,20.584118,8.113072e+05,1.438533e+06,NaN
228,01100102,Großer Tiergarten,01,01.01.2021,3943375.966,"POLYGON ((389869.327 5820074.331, 389869.304 5...","POLYGON ((389869.327 5820074.331, 389869.304 5...",30.0,10623,"POLYGON ((386918.412 5819016.265, 386946.512 5...",7.710680,2.273915,8.962462e+04,1.162344e+06,NaN


In [11]:
print(final_areas['area_intersect'].sum())
print(final_areas['area_PLRID'].sum())

865865001.6097763
8827118916.835014


In [15]:
# Export the data
# PLR_ID is the index in the 
#final_areas = final_areas[[ 'PLR_ID','plz', 'pct', 'area_intersect', 'area_PLRID']]
#final_areas.to_stata('C:/Users/laura/Desktop/Housing/SocialHousing/data/temp/areas_merged1_alternative.dta')

inter = inter[[ 'PLR_ID','plz','pct_plz', 'area_intersect', 'area_PLRID']]
inter.to_stata('C:/Users/laura/Desktop/Housing/SocialHousing/data/temp/areas_merged2_alternative.dta')

In [12]:
# Finding adjacent neighbors 
final_areas["neighbors"] = None 

In [ ]:
for index, country in gdf.iterrows():
    # get 'not disjoint' countries
    neighbors = gdf[~gdf.geometry.disjoint(country.geometry)].NAME.tolist()
     # remove own name of the country from the list
    neighbors = [ name for name in neighbors if country.NAME != name ]

    # add names of neighbors as NEIGHBORS value
    gdf.at[index, "neighbors"] = ", ".join(neighbors)

In [ ]:
final_areas.head()